In [ ]:
# Ref:
# YouTube Video
# Sam Witteveen
# Creating an AI Agent with LangGraph Llama 3 & Groq
# https://www.youtube.com/watch?v=lvQ96Ssesfk

In [ ]:
!pip install -U -q google-generativeai # Install the Python SDK
#!pip -q install groq
!pip -q install tavily-python

In [ ]:
!pip -q install -U sentence-transformers
!pip -q install faiss-gpu

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import re
import google.generativeai as genai
#from google.colab import userdata

In [ ]:
# The embeddings and the dataframe created and saved in Part 1

PATH_TO_EMBEDS = '../input/part-1-build-an-arxiv-rag-search-system-w-faiss/compressed_array.npz'
PATH_TO_DF = '../input/part-1-build-an-arxiv-rag-search-system-w-faiss/compressed_dataframe.csv.gz'

# Define the API clients

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()


In [ ]:
from tavily import TavilyClient

tavily_client = TavilyClient(api_key = user_secrets.get_secret("TAVILY_API_KEY"))

In [ ]:
import google.generativeai as genai

genai.configure(api_key = user_secrets.get_secret("GOOGLE_API_KEY"))



# What is the objective?

- Create an llm research assistant that has a chat interface


# Create a list of agents

AGENTS
1. chat_agent
2. router_agent
3. research_agent
4. final_answer_agent

TOOLS
- Function that returns an avaerage dog weight given a dog breed
- Calculate function that returns answers to calculations

BLOCKS
- ReAct block

# Helper functions

In [ ]:
def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)


In [ ]:
def create_message_history(system_message, user_input):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        },
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]

    return message_history



In [ ]:
def initialize_message_history(system_message):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        }
                    ]

    return message_history



## System messages

In [ ]:
comp_description = """

Competition Name: AI Mathematical Olympiad - Progress Prize 1

The goal of this competition is to create algorithms and models that can solve tricky math problems written in LaTeX format. Your participation will help to advance AI models’ mathematical reasoning skills and drive frontier knowledge.

The ability to reason mathematically is a critical milestone for AI. Mathematical reasoning is the foundation for solving many complex problems, from engineering marvels to intricate financial models. However, current AI capabilities are limited in this area.

The AI Mathematical Olympiad (AIMO) Prize is a new $10mn prize fund to spur the open development of AI models capable of performing as well as top human participants in the International Mathematical Olympiad (IMO).
This competition includes 110 problems similar to an intermediate-level high school math challenge. The Gemma 7B benchmark for these problems is 3/50 on the public and private test sets.

The assessment of AI models' mathematical reasoning skills faces a significant hurdle, the issue of train-test leakage. Models trained on Internet-scale datasets may inadvertently encounter test questions during training, skewing the evaluation process.

To address this challenge, this competition uses a dataset of 110 novel math problems, created by an international team of problem solvers, recognizing the need for a transparent and fair evaluation framework. The dataset encompasses a range of difficulty levels, from simple arithmetic to algebraic thinking and geometric reasoning. This will help to strengthen the benchmarks for assessing AI models' mathematical reasoning skills, without the risk of contamination from training data.

This competition offers an exciting opportunity to benchmark open AI models against each other and foster healthy competition and innovation in the field. By addressing this initial benchmarking problem, you will contribute to advancing AI capabilities and help to ensure that its potential benefits outweigh the risks.

Join us as we work towards a future where AI models’ mathematical reasoning skills are accurately and reliably assessed, driving progress and innovation across industries.
"""

In [ ]:
chat_agent_system_message = f"""
You are a helpful Kaggle competition research assistant.
Competition: {comp_description}

1. You provide polite answers to simple questions.
If the user's input requires only a simple answer, then output your answer as JSON.

Example session:

Question: Hello. How are you?

You output:

{{
"Answer": "I'm fine, thanks.",
"Status": "DONE"
}}

2. You can also run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
Output your response as a JSON string.

Your available actions are:

find_arxiv_research_papers:
e.g. find_arxiv_research_papers: [list of search keywords and phrases for a RAG search of the ArXiv database.]
Returns research papers from the ArXiv database.

run_web_search:
e.g. run_web_search: [list of search keywords and phrases for a web search]
Returns text content from search results.

You can only call one action at a time.

Example session:

Question: What are the latest techniques for detecting Pneumonia on x-rays using AI?
{{
"Thought": "I should look for relevant research papers in the ArXiv database by using find_arxiv_research_papers.",
"Action": {{"function":"find_arxiv_research_papers", "input": ["Pneumonia detection with AI", "Computer vision", "Object detection"]}},
"Status": "PAUSE"
}}

You will be called again with this:

Observation: <results>A list of research papers and their content</results>

You then output:
{{
"Answer": "Your final report.",
"Status": "DONE"
}}
""".strip()


# Set up the LLM

In [ ]:
def make_llm_api_call(message):

    """
    Makes a call to the Llama3 model on Groq.
    Args:
        message_history (List of dicts): The message history
    Returns:
        response_text: (str): The text response from the LLM
    """

    response = chat.send_message(message)
    
    text_response = response.text

    return text_response


# Example

model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction="You are a helpful assistant.",
)

chat = model.start_chat()

user_message = "What's your name?"

response = make_llm_api_call(user_message)

print(response)

## Set up the tools



### ArXiv RAG search tool

In [ ]:
def run_faiss_search(query_text, top_k):
    
    # Run FAISS exhaustive search
    
    query = [query_text]

    # Vectorize the query string
    query_embedding = sent_model.encode(query)

    # Run the query
    # index_vals refers to the chunk_list index values
    scores, index_vals = faiss_index.search(query_embedding, top_k)
    
    # Get the list of index vals
    index_vals_list = index_vals[0]
    
    return index_vals_list
    

def run_rerank(index_vals_list, query_text):
    
    chunk_list = list(df_data['prepared_text'])

    # Replace the chunk index values with the corresponding strings
    pred_strings_list = [chunk_list[item] for item in index_vals_list]

    # Format the input for the cross encoder
    # The input to the cross_encoder is a list of lists
    # [[query_text, pred_text1], [query_text, pred_text2], ...]

    cross_input_list = []

    for item in pred_strings_list:

        new_list = [query_text, item]

        cross_input_list.append(new_list)


    # Put the pred text into a dataframe
    df = pd.DataFrame(cross_input_list, columns=['query_text', 'pred_text'])

    # Save the orginal index (i.e. df_data index values)
    df['original_index'] = index_vals_list

    # Now, score all retrieved passages using the cross_encoder
    cross_scores = cross_encoder.predict(cross_input_list)

    # Add the scores to the dataframe
    df['cross_scores'] = cross_scores

    # Sort the DataFrame in descending order based on the scores
    df_sorted = df.sort_values(by='cross_scores', ascending=False)
    
    # Reset the index (*This was missed previously*)
    df_sorted = df_sorted.reset_index(drop=True)

    pred_list = []

    for i in range(0,len(df_sorted)):

        text = df_sorted.loc[i, 'pred_text']

        # Get the arxiv id
        # original_index refers to the index values in df_filtered
        original_index = df_sorted.loc[i, 'original_index']
        arxiv_id = df_data.loc[original_index, 'id']
        cat_text = df_data.loc[original_index, 'cat_text']
        title = df_data.loc[original_index, 'title']

        # Crete the link to the research paper pdf
        link_to_pdf = f'https://arxiv.org/pdf/{arxiv_id}'

        item = {
            'arxiv_id': arxiv_id,
            'link_to_pdf': link_to_pdf,
            'cat_text': cat_text,
            'title': title,
            'abstract': text
        }

        pred_list.append(item)

    return pred_list


def print_search_results(pred_list, num_results_to_print):
    
    for i in range(0,num_results_to_print):
        
        pred_dict = pred_list[i]
        
        link_to_pdf = pred_dict['link_to_pdf']
        abstract = pred_dict['abstract']
        cat_text = pred_dict['cat_text']
        title = pred_dict['title']

        print('Title:',title)
        print('Categories:',cat_text)
        print('Abstract:',abstract)
        print('Link to pdf:',link_to_pdf)
        print()
    
   
def run_arxiv_search(query_text, top_k=50):
    
    # Run a faiss greedy search
    pred_index_list = run_faiss_search(query_text, top_k)

    # This returns a list of dicts with length equal to top_k
    pred_list = run_rerank(pred_index_list, query_text)
    
    # Print the results
    #print_search_results(pred_list, num_results_to_print)
    
    return pred_list
    

In [ ]:
# Load the compressed array
embeddings = np.load(PATH_TO_EMBEDS)

# Access the array by the name you specified ('my_array' in this case)
embeddings = embeddings['array_data']

embeddings.shape

In [ ]:
# Load the compressed DataFrame

df_data = pd.read_csv(PATH_TO_DF, compression='gzip')

print(df_data.shape)

#df_data.head()

In [ ]:
# Initialize FAISS

import faiss

embed_length = embeddings.shape[1]

faiss_index = faiss.IndexFlatL2(embed_length)

# Add the embeddings to the index
faiss_index.add(embeddings)

faiss_index.is_trained

In [ ]:
# Initialize sentence_transformers

from sentence_transformers import SentenceTransformer

sent_model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
# Initialize the cross_encoder for reranking

from sentence_transformers import CrossEncoder

# We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
# Example

query_text = """

I want to build an invisibility cloak like the one in Harry Potter.

"""


# RUN THE SEARCH
num_results_to_print = 20 # top_k = 300
pred_list = run_arxiv_search(query_text, top_k=5)

### Tavily web search

In [ ]:
def run_tavily_search(query, num_results=50):

    """
    Uses the Tavily API to run a web search
    Args:
        query (str): The user query
        num_results (int): Num search results
    Returns:
        tav_response (json string): The search results in json format
    """

    # For basic search:
    tav_response = tavily_client.search(query=query, max_results=num_results)

    return tav_response


# Example

query = "How much does a bulldog weigh?"

results = run_tavily_search(query, num_results=2)

print(results)

# Set up the Agents

In [ ]:
def run_chat_agent(message):

    print("---CHAT AGENT---")

    # Prompt the llm
    response = make_llm_api_call(message)

    print(response)

    return response



# Example

model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction = chat_agent_system_message,
)

chat = model.start_chat()

message = "hello"

# Prompt the chat_agent
response = run_chat_agent(message)

response

In [ ]:
def run_router_agent(llm_response):

    """
    Route to web search or not.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """

    print("---ROUTER AGENT---")


    # Extract the status
    json_response = json.loads(llm_response)
    status = json_response['Status']
    #status = extract_json_str_value(response, "Status").strip()

    print("Status:", status)

    if status == 'PAUSE':
        print("Route: to_research_agent")
        return "to_research_agent"

    elif status == 'DONE':
        print("Route: to_final_answer")
        return "to_final_answer"



# Example

model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction = chat_agent_system_message,
)

chat = model.start_chat()

message = "hello"

# Prompt the chat_agent
response = run_chat_agent(message)

# Run router_agent
route = run_router_agent(response)

route

In [ ]:
def run_research_agent(llm_response):

    print("---RESEARCH AGENT---")

    # Extract the status
    json_response = json.loads(llm_response)
    action_dict = json_response['Action']
    func_to_run = action_dict['function']
    func_input_list = action_dict['input']
    
    answer_list = []

    if func_to_run == "find_arxiv_research_papers":
        for search_query in func_input_list:
            answer = run_arxiv_search(search_query, top_k=50)
            answer_list.append(answer)
    else:
        for search_query in func_input_list:
            answer = run_tavily_search(search_query, num_results=50)
            answer_list.append(answer)

    print("func_to_run:", func_to_run)
    print("func_arg:", func_input_list)
    print("Output:", answer_list)

    return answer_list



# Example

model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction = chat_agent_system_message,
)

chat = model.start_chat()

message = "What are some of the state of the art techniques that I can use to solve this problem?"


# Prompt the chat_agent
response = run_chat_agent(message)

# Run router_agent
route = run_router_agent(response)


if route == "to_research_agent":
    answer = run_research_agent(response)

    # Update message history
    #message = {"role": "user", "content": f"Observation: {answer}"}
    #message_history.append(message)



In [ ]:
def run_final_answer_agent(llm_response):

    print("---FINAL ANSWER AGENT---")

    json_response = json.loads(llm_response)
    final_answer = json_response['Answer']

    print("Final answer:", final_answer)

# Run the system

In [ ]:
model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction = chat_agent_system_message,
)

chat = model.start_chat()


message = "What is the current state of the art for this subject?"

for i in range(0,10):

    # Prompt the chat_agent
    llm_response = run_chat_agent(message)

    # Run router_agent
    route = run_router_agent(llm_response)


    if route == "to_research_agent":

        answer = run_research_agent(llm_response)
        
        user_input = f"Observation: {answer}"

    else:

        run_final_answer_agent(llm_response)

        break



# Run infinitely with a user input field

In [ ]:
model = genai.GenerativeModel(
    "models/gemini-1.5-flash",
    system_instruction = chat_agent_system_message,
)

chat = model.start_chat()


while True:

    print()
    print("==========")
    user_input = input("Enter something (or 'q' to quit): ")
    print("==========")
    
    i = i + 1

    if user_input.lower() == 'q':
        print("Exiting the loop. Goodbye!")
        break  # Exit the loop


    for i in range(0,10):

        # Prompt the chat_agent
        llm_response = run_chat_agent(user_input)

        # Run router_agent
        route = run_router_agent(llm_response)


        if route == "to_research_agent":

            answer = run_research_agent(llm_response)
            
            user_input = f"Observation: {answer}"

        else:

            run_final_answer_agent(llm_response)

            break

